# Path

In [16]:
# Global path
GLOBAL_PATH = "C:/Users/rayji/OneDrive/Documents/Projet 2 ML"

# GloVe
GLOVE_PATH = f"{GLOBAL_PATH}/data/glove.twitter.27B.100d.txt"

# Train full
TRAIN_NEG_FULL_PATH = f"{GLOBAL_PATH}/data/train_neg_full.txt"
TRAIN_POS_FULL_PATH = f"{GLOBAL_PATH}/data/train_pos_full.txt"

# Train
TRAIN_NEG_PATH = f"{GLOBAL_PATH}/data/train_neg.txt"
TRAIN_POS_PATH = f"{GLOBAL_PATH}/data/train_pos.txt"

# Test
TEST_PATH = f"{GLOBAL_PATH}/data/test_data.txt"

# Preprocessed data
TRAIN_PREP_PATH = f"{GLOBAL_PATH}/data/preprocessed/train_gru.csv"
TEST_PREP_PATH = f"{GLOBAL_PATH}/data/preprocessed/test_gru.csv"

# Weight
WEIGHT_PATH = f"{GLOBAL_PATH}/weight"

# Abstract method

In [2]:
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import pandas as pd

In [3]:
class AbstractModel(ABC):
    def __init__(self, weights_path: str):
        self.__weights_path = weights_path


    @abstractmethod
    def get_preprocessing_methods(self, is_test: bool = False):
        pass


    @abstractmethod
    def fit_predict(self, X, y, ids_test, X_test, prediction_path):
        pass


    @abstractmethod
    def predict(self, ids, X, path):
        pass


    @staticmethod
    def _create_submission(ids: list[int], predictions: list[int], path: str):
        # Generating the submission file
        submission = pd.DataFrame(columns=["Id", "Prediction"],
                                data={"Id": ids, "Prediction": predictions})

        # For many models the labels are 0 or 1. Replacing 0s with -1s.
        submission["Prediction"].replace(0, -1, inplace=True)

        # Saving the file
        submission.to_csv(path, index=False)


    @staticmethod
    def _split_data(X: pd.DataFrame, y: pd.DataFrame, test_size: float = 0.2, random_state: int = 42, **kwargs) -> tuple:
        print("Splitting data in train and test set...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, **kwargs)

        return X_train, X_test, y_train, y_test

# Preprocessing


In [6]:
!pip install symspellpy

     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
      --------------------------------------- 0.1/2.6 MB 825.8 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/2.6 MB 1.2 MB/s eta 0:00:03
     ----- ---------------------------------- 0.4/2.6 MB 2.3 MB/s eta 0:00:01
     ------------ --------------------------- 0.8/2.6 MB 4.1 MB/s eta 0:00:01
     ------------------- -------------------- 1.3/2.6 MB 5.1 MB/s eta 0:00:01
     ----------------------------- ---------- 1.9/2.6 MB 6.4 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 7.5 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.2 kB ? eta -:--:--
     ---------------------------------------- 57.2/57.2 kB ? eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with

  error: subprocess-exited-with-error
  
  × Building wheel for editdistpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      <string>:23: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      running bdist_wheel
      running build
      running build_ext
      building 'editdistpy.levenshtein' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for editdistpy
ERROR: Could not build wheels for editdistpy, which is required to install pyproject.toml-based projects


In [4]:
import pkg_resources
import nltk
import re
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell # Fuzzy search and word correction

In [5]:
# nltk weights
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [110]:
import importlib
import preprocessing
from preprocessing import *
# Reload the library
importlib.reload(preprocessing)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


<module 'preprocessing' from 'c:\\Users\\rayji\\OneDrive\\Documents\\GitHub\\ml-project-2-big-three\\preprocessing.py'>

In [19]:
def run_preprocessing(
    csr: AbstractModel, train_preprocessed_path=TRAIN_PREP_PATH, test_preprocessed_path=TEST_PREP_PATH, full_data=True
):
    """
    Runs the preprocessing methods according to the chosen classifier
      on the train and test data

    :param csr: chosen classifier (child of AbstractModel)
    :type csr: AbstractModel
    :param train_preprocessed_path: path to load train data
    :type train_preprocessed_path: str
    :param test_preprocessed_path: path to load test data
    :type test_preprocessed_path: str
    :param full_data: if False, the small dataset (200K rows) is used
    :type full_data: bool, optional
    """

    # Read data
    if full_data:
        dataset_files = [TRAIN_NEG_FULL_PATH, TRAIN_POS_FULL_PATH]
    else:
        dataset_files = [TRAIN_NEG_PATH, TRAIN_POS_PATH]

    train_preprocessing = Preprocessing(dataset_files, submission=False)
    test_preprocessing = Preprocessing([TEST_PATH], submission=True)

    # Preprocess it
    for method in csr.get_preprocessing_methods(istest=False):
        getattr(train_preprocessing, method)()

    for method in csr.get_preprocessing_methods(istest=True):
        getattr(test_preprocessing, method)()

    # Save it
    train_df = train_preprocessing.get()
    train_df = train_df.sample(frac=1)

    train_df.to_csv(train_preprocessed_path, index=False)
    test_preprocessing.get().to_csv(test_preprocessed_path, index=False)


In [111]:
def run_preprocessing(
    train_preprocessed_path=TRAIN_PREP_PATH, test_preprocessed_path=TEST_PREP_PATH, full_data=True
):
    """
    Runs the preprocessing methods according to the chosen classifier
      on the train and test data

    :param csr: chosen classifier (child of AbstractModel)
    :type csr: AbstractModel
    :param train_preprocessed_path: path to load train data
    :type train_preprocessed_path: str
    :param test_preprocessed_path: path to load test data
    :type test_preprocessed_path: str
    :param full_data: if False, the small dataset (200K rows) is used
    :type full_data: bool, optional
    """

    # Read data
    if full_data:
        dataset_files = [TRAIN_NEG_FULL_PATH, TRAIN_POS_FULL_PATH]
    else:
        dataset_files = [TRAIN_NEG_PATH, TRAIN_POS_PATH]

    train_preprocessing = Preprocessing(dataset_files, is_test=False)
    test_preprocessing = Preprocessing([TEST_PATH], is_test=True)

    # Preprocess it

    methods = []

    methods.extend([
    'drop_duplicates',
    'remove_parentheses',
    'remove_parentheses',
    'remove_parentheses',
    'remove_selected_characters',
    'remove_tag',
    'correct_spacing_indexing',
    'remove_space_between_emoticons',
    'correct_spacing_indexing',
    'hashtags_to_tags',
    'slang_to_word',
    'word_segmentation',
    'correct_spelling',
    'lemmatize',
    'remove_stopwords',
    'correct_spacing_indexing'
    ])

    for method in methods:
        getattr(train_preprocessing, method)()

    methods.append('drop_duplicates')

    for method in methods:
        getattr(test_preprocessing, method)()

    train_df = train_preprocessing.__get__()
    #train_df.to_csv(train_preprocessed_path, index=False)

    test_df = test_preprocessing.__get__()
    #test_df.to_csv(test_preprocessed_path, index=False)

    return train_df, test_preprocessing

In [112]:
train_preprocessed_dataset, test_preprocessed_dataset = run_preprocessing(full_data=False)

Executing: `drop_duplicates`
Executing: `remove_parentheses`
Executing: `remove_parentheses`
Executing: `remove_parentheses`
Executing: `remove_selected_characters`
Removing selected characters...
Executing: `remove_tag`
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `remove_space_between_emoticons`
Removing space between emoticons...
Executing: `correct_spacing_indexing`
Correcting spacing...
Executing: `hashtags_to_tags`
Converting hashtags to tags...
Executing: `slang_to_word`


100%|██████████| 181321/181321 [00:00<00:00, 479377.36it/s]


Executing: `word_segmentation`


 47%|████▋     | 85172/181321 [08:00<09:34, 167.45it/s]

In [105]:
train_preprocessed_dataset.to_csv('C:/Users/rayji/OneDrive/Documents/GitHub/ml-project-2-big-three/data/preprocessed/train_preprocessed.csv', index=False)

In [27]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import words
from collections import Counter


# Download the English language dictionary
nltk.download('words')
nltk.download('punkt')


texts = train_preprocessed_dataset['text']

# Function to clean and tokenize text
def clean_and_tokenize(text):
    # Remove non-alphabetic characters (keep only letters, parentheses, numbers, '.', '-', and '!')
    clean_text = re.sub(r'[^a-zA-Z0-9.!\-()]', ' ', text)
    # Tokenize
    return word_tokenize(clean_text.lower())

# Tokenize the text
tokens = [clean_and_tokenize(text) for text in texts]
flat_tokens = [item for sublist in tokens for item in sublist]


# Load a set of standard English words
english_words = set(words.words())

# Filter out standard English words
non_standard_tokens = [word for word in flat_tokens if word not in english_words]

# Count the frequencies
word_freq = Counter(non_standard_tokens)

# You can define a threshold for what you consider 'high frequency'
high_freq_threshold = 10  # Example threshold
high_freq_words = {word: count for word, count in word_freq.items() if count > high_freq_threshold}

ordered_words = {word: count for word, count in sorted(high_freq_words.items(), key=lambda item: item[1], reverse=True)}
print(ordered_words)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rayji\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'url': 18372, 'urls': 11089, 'xxx': 5315, 'hama': 4465, 'll': 3968, 'fuck': 3024, 'cd': 2247, 've': 2148, 'okay': 2081, 'fucking': 1834, 'shit': 1545, 'pm': 1457, 'kid': 1197, 'hardcover': 1091, 'friday': 1015, 'pc': 971, 'int': 852, 'anymore': 839, 'boyfriend': 790, 'favourite': 782, 'sarah': 733, 'laptop': 690, 'hmm': 664, 'email': 595, 'proud': 589, 'saturday': 586, 'liam': 583, 'co': 534, 'american': 517, 'yamaha': 517, 'facebook': 512, 'goodnight': 508, 'justin': 504, 'english': 504, 'ipod': 479, 'mini': 466, 'bro': 465, 'hang': 453, 'oof': 451, 'prom': 445, 'eth': 437, 'der': 404, 'online': 397, 'nigga': 391, 'monday': 381, 'sunday': 376, 'kinda': 374, 'london': 364, 'thursday': 364, 'blog': 356, 'etc': 340, 'congrats': 338, 'girlfriend': 333, 'ohio': 315, 'ii': 315, 'iphone': 314, 'manuf': 306, 'thurs': 301, 'april': 296, 'jus': 290, 'french': 277, 'compaq': 273, 'hun': 261, 'louis': 245, 'fax': 244, 'misc': 244, 'america': 244, 'john': 240, 'nah': 236, 'internet': 236, 'app': 

# Old

In [10]:
import pandas as pd
import importlib
import utils 
from utils import *
from os import listdir
from os.path import isfile, join
from random import shuffle
# Reload the library
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\rayji\\OneDrive\\Documents\\GitHub\\ml-project-2-big-three\\utils.py'>

In [12]:
data_train = load_data()
test_data = load_test_data()

In [13]:
# Print the number of lines in each dataset
print(data_train.shape)
print(test_data.shape)

(200000, 2)
(10000, 1)


In [5]:
# Use read_pickle() to open the .pkl file
#vocab = pd.read_pickle('vocab.pkl')
#cooc_matrix = pd.read_pickle('cooc.pkl')

In [6]:
import scipy.sparse

# Assuming 'sparse_matrix' is your sparse matrix
# Replace row_index and col_index with the specific indices you want to access
row_index = 0
col_index = 0

# Access the element at the specified row and column indices
row = cooc_matrix.getrow(row_index)
element = row[0, col_index]

# 'element' now contains the value at the specified position in the sparse matrix
element

NameError: name 'cooc_matrix' is not defined

In [15]:
data_train

,text,label
0,<user> i dunno justin read my mention or not ....,1
1,"because your logic is so dumb , i won't even c...",1
2,""" <user> just put casper in a box ! "" looved t...",1
3,<user> <user> thanks sir > > don't trip lil ma...,1
4,visiting my brother tmr is the bestest birthda...,1
...,...,...
199995,can't wait to fake tan tonight ! hate being pale,0
199996,<user> darling i lost my internet connection ....,0
199997,kanguru defender basic 4 gb usb 2.0 flash driv...,0
199998,rizan is sad now,0


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_train['text'], data_train['label'], test_size=0.01, random_state=42)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the text data into TF-IDF features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression(max_iter=5000)
model.fit(X_train_tfidf, y_train)

# Predict labels for the test set
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.808


In [17]:
y_test_pred = model.predict(vectorizer.transform(test_data['text']))

In [39]:
export_submission(y_test_pred)